In [6]:
import math
from sklearn import neighbors
import os
import pickle
import cv2
import gabor
from tqdm import tqdm
import numpy as np
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

In [15]:
def train_rf(train_dir, model_save_path=None):
    X = []
    y = []
    
    for class_dir in tqdm(os.listdir(train_dir)):
        for img_path in os.listdir(train_dir + "/" + class_dir):
            # image = cv2.imread(train_dir + "/" + class_dir + "/" + img_path)
            image = train_dir + "/" + class_dir + "/" + img_path
            if "_l_" in img_path:
                H = gabor.process(image, "l")
            else: 
                H = gabor.process(image, "r")
            
            # Add face encoding for current image to training set
            X.append(H)
            y.append(class_dir)
    
    X = np.array(X)
    
    rf_clf = RandomForestClassifier(n_estimators=150, n_jobs=-1, random_state=42)

    rf_clf.fit(X, y)
    
    if model_save_path is not None:
        with open(model_save_path, 'wb') as f:
            pickle.dump(rf_clf, f)

    return rf_clf

In [16]:
def train_svm(train_dir, model_save_path=None):
    X = []
    y = []
    
    for class_dir in tqdm(os.listdir(train_dir)):

        for img_path in os.listdir(train_dir + "/" + class_dir):
            # image = cv2.imread(train_dir + "/" + class_dir + "/" + img_path)
            image = train_dir + "/" + class_dir + "/" + img_path
            if "_l_" in img_path:
                H = gabor.process(image, "l")
            else: 
                H = gabor.process(image, "r")
            
            # Add face encoding for current image to training set
            X.append(H)
            y.append(class_dir)
    
    X = np.array(X)

    svm_clf = SVC(kernel='linear', probability=True)
    svm_clf.fit(X, y)
    
    if model_save_path is not None:
        with open(model_save_path, 'wb') as f:
            pickle.dump(svm_clf, f)
    
    return svm_clf

In [17]:
def train_knn(train_dir, model_save_path=None,  n_neighbors=None, knn_algo='ball_tree', verbose=False):
    X = []
    y = []
    
    for class_dir in tqdm(os.listdir(train_dir)):

        for img_path in os.listdir(train_dir + "/" + class_dir):
            image = train_dir + "/" + class_dir + "/" + img_path
            if "_l_" in img_path:
                H = gabor.process(image, "l")
            else: 
                H = gabor.process(image, "r")
            
            X.append(H)
            y.append(class_dir)
            
    if n_neighbors is None:
        n_neighbors = int(round(math.sqrt(len(X))))
        if verbose:
            print("Chose n_neighbors automatically:", n_neighbors)
        
    X = np.array(X)

    knn_clf = neighbors.KNeighborsClassifier(n_neighbors=n_neighbors, algorithm=knn_algo, weights='distance')
    knn_clf.fit(X, y)
    
    if model_save_path is not None:
        with open(model_save_path, 'wb') as f:
            pickle.dump(knn_clf, f)
    
    return knn_clf

In [18]:
def predict(X_img_path, clf=None, model_path=None):
    if clf is None:
        with open(model_path, 'rb') as f:
            clf = pickle.load(f)
            
    faces_encodings = gabor.process(X_img_path)

    # print(faces_encodings.shape)
    faces_encodings = np.array(faces_encodings)
    faces_encodings = faces_encodings.reshape(1, -1)

    # print(svm_clf.predict(faces_encodings))
    
    return clf.predict(faces_encodings)

In [19]:
def predict(X_img_path, model=None, model_path=None):
    if model is None:
        if model_path is None:
            raise ValueError("Either a preloaded model or a model path must be provided.")
        with open(model_path, 'rb') as f:
            model = pickle.load(f)

    # Process the image to extract features (assuming Gabor feature extraction)
    faces_encodings = gabor.process(X_img_path)

    # Ensure the features are in the correct shape for prediction
    faces_encodings = np.array(faces_encodings)
    faces_encodings = faces_encodings.reshape(1, -1)
    
    # Return the predictions from the model
    return model.predict(faces_encodings)


In [20]:
print("Training random forest...")
classifier = train_rf("./train_ver2", model_save_path="trained_rf_model.clf")
print("Training complete!")

Training random forest...


100%|██████████| 312/312 [02:36<00:00,  1.99it/s]


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (4883, 2) + inhomogeneous part.

In [ ]:
# print("Training SVM...")
# classifier = train_svm("./train_ver2", model_save_path="trained_svm_model.clf")
# print("Training complete!")

In [39]:
print("Training KNN...")
classifier = train_knn("./train_ver2", model_save_path="trained_knn_model.clf", n_neighbors=1)
print("Training complete!")

Training KNN...


100%|██████████| 312/312 [02:38<00:00,  1.97it/s]


Training complete!


In [40]:
correct_prediction_rf = 0
# correct_prediction_svm = 0
correct_prediction_knn = 0
print(len(os.listdir('./test_ver2')))
for image_dir in tqdm(os.listdir('./test_ver2')):
    full_file_path = './test_ver2/' + image_dir
    
    predictions_rf = predict(full_file_path, model_path="trained_rf_model.clf")
    # predictions_svm = predict(full_file_path, model_path="trained_svm_model.clf")
    predictions_knn = predict(full_file_path, model_path="trained_knn_model.clf")
    
    y_test_rf = str(predictions_rf[0])
    y_pred_rf = image_dir[:4]
    if y_test_rf == y_pred_rf:
        correct_prediction_rf += 1
    # else:
    #     print("Looking for palm print in {}".format(image_dir))
    #     print("Real prediction: ", predictions_rf)

    # y_test_svm = str(predictions_svm[0])
    # y_pred_svm = image_dir[:4]
    # if y_test_svm == y_pred_svm:
    #     correct_prediction_svm += 1
    # else:
    #     print("Looking for palm print in {}".format(image_dir))
    #     print("Real prediction: ", predictions_svm)

    y_test_knn = str(predictions_knn[0])
    y_pred_knn = image_dir[:4]
    if y_test_knn == y_pred_knn:
        correct_prediction_knn += 1
    # else:
    #     print("Looking for palm print in {}".format(image_dir))
    #     print("Real prediction: ", predictions_knn)
    
    
accuracy_rf = correct_prediction_rf / len(os.listdir('./test_ver2'))
# accuracy_svm = correct_prediction_svm / len(os.listdir('./test_ver2_1'))
accuracy_knn = correct_prediction_knn / len(os.listdir('./test_ver2'))
print(f"Accuracy KNN: {accuracy_knn}\nAccuracy RF: {accuracy_rf}\nAccuracy SVM: 0\n")

619


100%|██████████| 619/619 [11:48<00:00,  1.14s/it]

Accuracy KNN: 0.8949919224555735
Accuracy RF: 0.9030694668820679
Accuracy SVM: 0

